In [1]:
import glob
import numpy as np
import pandas as pd

weather_path = 'yvr-weather'

weather_files = glob.glob(weather_path + '/*.csv')

weather = []
for file in weather_files:
    df = pd.read_csv(file, skiprows=16)
    weather.append(df)

weather = pd.concat(weather).reset_index()

In [2]:
weather = weather[['Date/Time', 'Weather']]
weather = weather[weather['Weather'].notnull()].reset_index()
weather['datetime'] = pd.to_datetime(weather['Date/Time'])

In [3]:
katkam_path = 'katkam-rescaled'
katkam_files = glob.glob(katkam_path + '/*.jpg')

In [4]:
katkam_df = pd.DataFrame(katkam_files)
katkam_df.columns = ['filename']

In [5]:
katkam_df = pd.DataFrame(katkam_df['filename'].str.extract('-(\d{14})\.jpg', expand=False))

In [6]:
katkam_df.columns = ['datetime']

In [7]:
katkam_df['datetime'] = pd.to_datetime(katkam_df['datetime'])

In [8]:
weather = weather[['Weather', 'datetime']]

In [9]:
weather.columns = [['weather', 'datetime']]

In [10]:
merged = weather.merge(katkam_df, on='datetime')

In [12]:
merged['weather'].unique()

array(['Rain', 'Cloudy', 'Rain Showers', 'Mostly Cloudy', 'Rain,Fog',
       'Mainly Clear', 'Clear', 'Moderate Rain,Fog',
       'Moderate Rain Showers', 'Moderate Rain Showers,Fog', 'Drizzle',
       'Rain,Drizzle', 'Snow', 'Moderate Snow', 'Rain,Snow',
       'Snow Showers', 'Heavy Rain,Fog', 'Drizzle,Fog', 'Moderate Rain',
       'Rain Showers,Fog', 'Rain,Drizzle,Fog', 'Fog', 'Snow,Fog',
       'Freezing Fog', 'Rain,Snow,Fog', 'Thunderstorms',
       'Rain Showers,Snow Showers,Fog', 'Rain Showers,Snow Pellets',
       'Rain Showers,Snow Showers', 'Moderate Rain,Drizzle', 'Heavy Rain'], dtype=object)

In [15]:
merged.groupby('weather').count()

,datetime
weather,
Clear,131
Cloudy,470
Drizzle,16
"Drizzle,Fog",8
Fog,12
Freezing Fog,2
Heavy Rain,1
"Heavy Rain,Fog",2
Mainly Clear,344


In [19]:
merged['rain'] = merged['weather'].str.contains('Rain|Drizzle')

In [20]:
merged['snow'] = merged['weather'].str.contains('Snow')

In [21]:
merged['clear'] = merged['weather'].str.contains('Clear')
merged['fog'] = merged['weather'].str.contains('Fog')
merged['cloudy'] = merged['weather'].str.contains('Cloud')

In [25]:
merged = merged[merged['clear'] != merged['cloudy']]

In [27]:
merged[['datetime', 'rain', 'snow', 'clear', 'fog', 'cloudy']].to_csv('cleaned_data_v2.csv', index=False)